In [48]:
def load__data(file_path):
    data,labels,words,cw,tags = [],[],[],[],[]
    tag2idx={'B-A': 1, 'I-A': 2, 'O': 0}
    fh = open(file_path)
    for line in fh:
        line = line.strip()
        if line=='':
            cw.append(words)
            #Sentence ended.
            if len(tags) <70:
              tags+=[0]*(70-len(tags))
            if len(words)<70:
              words+=['0']*(70-len(words))
            data.append(words)
            labels.append(tags)
            words,tags = [],[]
        else:
            word, tag = line.split("\t")
            words.append(word)
            tags.append(tag2idx[tag])
    fh.close()
    return data,cw,labels

test_path = '/content/drive/MyDrive/Colab Notebooks/AE/AE_Datasets/Restaurants_Test_Gold_mod.iob'
train_path = '/content/drive/MyDrive/Colab Notebooks/AE/AE_Datasets/Restaurants_Train_v2_mod.iob'

train,c1,train_labels = load__data(train_path)
test,c2,test_labels = load__data(test_path)
c=c1+c2
train+=test
train_labels+=test_labels

In [30]:
import pandas as pd
df=pd.DataFrame(train)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,But,the,staff,was,so,horrible,to,us,0,0,...,0,0,0,0,0,0,0,0,0,0
1,To,be,completely,fair,the,only,redeeming,factor,was,the,...,0,0,0,0,0,0,0,0,0,0
2,The,food,is,uniformly,exceptional,with,a,very,capable,kitchen,...,0,0,0,0,0,0,0,0,0,0
3,Where,Gabriela,personaly,greets,you,and,recommends,you,what,to,...,0,0,0,0,0,0,0,0,0,0
4,For,those,that,go,once,and,dont,enjoy,it,all,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#!pip install flair
# Extracting POST for aspects
import pandas as pd
import numpy as np
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load("flair/upos-english-fast")
train=np.array(train)
train_labels=np.array(train_labels)
df=pd.DataFrame(columns=["sent","B-A","I-A"])
AI={}
AB={}
index=[]
Aspects=[]
for i,x in enumerate(train[0:12]):
  sentence = Sentence(" ".join(c[i]))
  b=np.where(np.array(train_labels[i]) == 1)
  I=np.where(np.array(train_labels[i]) == 2)
  if len(I[0])==0:
    continue
  for x in b:
    try:
      j=x[0]
    except:
      break
    aspect= train[i][j]
    j+=1
    while ( j in I[0]):
      aspect+=train[i][j]
    Aspects.append(aspect)
  tagger.predict(sentence)
  tags=np.array([ str(k).split()[-2] for k in sentence.get_labels('pos')])
  tags1=tags[I]
  tags2=tags[b]
  index.append(i)
  for y in tags1:
    if y in AI.keys():
     AI[y]+=1
    else:
     AI[y]=1
  for y in tags2:
    if y in AB.keys():
     AB[y]+=1
    else:
     AB[y]=1
  df.loc[len(df.index)] = [c[i], np.array(x)[b], np.array(x)[I]]
df.head()

In [54]:
0 in [len(np.where(np.array(train_labels[4]) == 1)[0])]

True

In [57]:
# Extracting Aspects with more than one words
import pandas as pd
import numpy as np
index=[]
Aspects=[]
for i,x in enumerate(train):
  sentence = Sentence(" ".join(c[i]))
  b=np.where(np.array(train_labels[i]) == 1)
  I=np.where(np.array(train_labels[i]) == 2)
  if len(I[0])==0 or len(b[0])==0:
    continue
  index.append(i)
  for x in b:
    j=x[0]
    aspect=train[i][j]
    j+=1
    while ( j in I[0]):
      aspect+=' '+train[i][j]
      j+=1
    if len(aspect.split())>1:
       Aspects.append(aspect)
Aspects

['orrechiete with sausage and chicken',
 'interior decoration',
 'seltzer with lime',
 'eat family style',
 'interior decor',
 'fried rice',
 'indian food',
 'broth with noodles',
 'wine list',
 'Thai food',
 'glass of wine',
 'lasagnette appetizer',
 'beer selection',
 'Wine list selection',
 'sake martini',
 'garden terrace',
 'grilled branzino',
 'wait staff',
 'pretheater menu',
 'lobster sandwich',
 'Spider Roll',
 'Deep Fried Skewers',
 'rice dishes',
 'tuna tartar appetizer',
 'Indian food',
 'baby pizzas',
 'lava cake dessert',
 'baby pizzas',
 'selection of wines',
 'Italian decor',
 'dining experience',
 'wine selection',
 'panang duck',
 'BBQ Salmon',
 'Dal Bukhara',
 'glass of Leaping Lizard',
 'chinese food',
 'beverage manager',
 'tandoori salmon',
 'three course meal',
 'crawfish boiled',
 'Pad Thai',
 'values for your money',
 'wine by the glass',
 'Change Mojito',
 'soup for the udon',
 'plain pizza',
 'getting a table',
 'takeout menu',
 'Thai food',
 'bottles of wine

In [51]:
# index: Indexes of aspects consisting of more than one word
print('Counts:{}'.format(len(index)))
print(index)
df.tail(n=50)

Counts:945
[7, 15, 25, 27, 30, 31, 40, 47, 49, 50, 51, 53, 56, 58, 59, 68, 71, 76, 81, 92, 104, 105, 107, 113, 120, 125, 126, 128, 129, 132, 133, 134, 135, 140, 141, 145, 146, 148, 149, 151, 152, 161, 164, 168, 169, 175, 176, 184, 186, 187, 194, 197, 198, 203, 207, 219, 220, 225, 229, 231, 234, 238, 241, 245, 251, 252, 253, 257, 258, 261, 268, 274, 278, 281, 282, 283, 284, 292, 293, 297, 301, 304, 313, 323, 329, 331, 335, 337, 342, 346, 348, 358, 360, 379, 387, 390, 391, 392, 403, 406, 409, 413, 414, 415, 416, 423, 432, 434, 441, 442, 447, 449, 451, 454, 457, 461, 466, 476, 488, 499, 501, 506, 507, 512, 513, 514, 516, 518, 525, 527, 533, 537, 538, 539, 540, 546, 548, 554, 560, 574, 576, 588, 592, 600, 603, 604, 615, 617, 626, 628, 630, 632, 635, 641, 642, 652, 660, 661, 663, 666, 669, 674, 676, 681, 684, 685, 686, 693, 696, 698, 699, 702, 704, 707, 709, 715, 716, 720, 729, 732, 734, 735, 737, 743, 745, 747, 752, 755, 761, 770, 778, 779, 780, 784, 787, 788, 789, 790, 801, 805, 806, 814,

,sent,B-A,I-A


In [52]:
train[7]

['Our',
 'agreed',
 'favorite',
 'is',
 'the',
 'orrechiete',
 'with',
 'sausage',
 'and',
 'chicken',
 'usually',
 'the',
 'waiters',
 'are',
 'kind',
 'enough',
 'to',
 'split',
 'the',
 'dish',
 'in',
 'half',
 'so',
 'you',
 'get',
 'to',
 'sample',
 'both',
 'meats',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0']

IndexError: ignored

tag 	meaning

ADJ 	adjective

ADP 	adposition

ADV 	adverb

AUX 	auxiliary

CCONJ 	coordinating conjunction

DET 	determiner

INTJ 	interjection

NOUN 	noun

NUM 	numeral

PART 	particle

PRON 	pronoun

PROPN 	proper noun

PUNCT 	punctuation

SCONJ 	subordinating conjunction

SYM 	symbol

VERB 	verb

X 	other